<h1>What is asynchronous Programming?</h1>

Asychronous Programming is a technique that allows execution of multiple tasks in <strong>concurrent</strong> without blocking the main thread of execution.

Async/Await are the keywords around which async programming revolves around.

Async Keyword enables us to perform async programming using await. Without async, await can not be used.

Whenever a task is <strong>awaited</strong>, the current thread of execution is freed and it can perform another operation it is handed over with. <br>

<h3>What is a Task?</h3> <br> 



        TIP: 
- Do NOT return void from async method. Just return Task. <br>
- Even if you don't await the task explicitly (await task), the task still runs because when you start a task using Task.Run, it begins execution immediately in the background. <br> 
Convention -> append Async on any method returning asynchronously.

        HOW TO MEASURE THE TIME A TASK TAKES TO COMPLETE.
<strong>
var watch = System.Diagnostics.Stopwatch.StartNew();    <br>
// Write your time consuming logic here <br>
var elapses = watch.ElapsedMilliseconds; <br>
Console.Write(elapses); <br>
</strong>

In [1]:
using System.Threading;

In [2]:
public void PerformWork(){
    int count = 1;
    for(long i = 0 ; i < 1000000000 ; i++) // Takes around 2 seconds
    {
        count++;
    }
    Console.WriteLine("Work has been done!");
}

In [3]:
PerformWork();
for(int i = 0 ; i < 5  ; i++)
{
    System.Threading.Thread.Sleep(1000);
    Console.WriteLine("I am main task");
}

Work has been done!
I am main task
I am main task
I am main task
I am main task
I am main task


In [4]:
await Task.Run(()=> PerformWork());
for(int i = 0 ; i < 10  ; i++)
{
    System.Threading.Thread.Sleep(1000);
    Console.WriteLine("I am main task");
}

Work has been done!
I am main task
I am main task
I am main task
I am main task
I am main task
I am main task
I am main task
I am main task
I am main task
I am main task


In [5]:
public async Task<string> PerformWorkAsync(){
    await Task.Delay(5000); // Simulates time consuming operation
    Console.WriteLine("Time consuming task is done");
    return "454684351";
}

In [6]:
using System.Threading;
string code = await PerformWorkAsync();
Console.WriteLine($"Code : {code}");
Console.WriteLine("Main Thread Executed");
Thread.Sleep(7000); // Simulate Time Consuming

Time consuming task is done
Code : 454684351
Main Thread Executed


<h1>        Running Multiple Tasks at once<h1>

Task is a special fundamental Type in Task Parallel Library (TPL) that represents an asynchronous operation. 

        How to define a Task?
Task.Run(async () => <br>
{ <br>
   // Perform some action here <br>
});<br>



In [7]:
Task task1  = Task.Run(async () =>
{
    for(int i =0 ; i < 10 ; i++)
    {
        await Task.Delay(1000); // Simulate time consuming Task
        Console.WriteLine("Task 1 executed");
    }
});


Task task2 = Task.Run(async () =>
{
    for(int i =0 ; i < 10 ; i++)
    {
        await Task.Delay(2000); // Simulate time consuming Task
        Console.WriteLine("Task 2 executed");
    }
});

In [8]:
await Task.WhenAll(task1,task2);
Console.WriteLine("Both methods executed");

Both methods executed


<h1>Cancelling the Task before it reaches completion</h1>

        Create a CancellationTokenSource:
 First, create a CancellationTokenSource object. This object generates cancellation tokens that can be passed to tasks. <br>
        
        Pass the CancellationToken to the Task: 
When starting an asynchronous operation, pass the cancellation token from the CancellationTokenSource to the task as a parameter.
        
        Check for Cancellation: 
Within the task's code, periodically check if cancellation has been requested by calling the ThrowIfCancellationRequested method on the cancellation token. If cancellation is requested, exit the operation as gracefully as possible.
        
        Cancelling the Task: 
To cancel the task, call the Cancel method on the CancellationTokenSource object.


In [9]:
static async Task PerformAsyncOperation(CancellationToken cancellationToken)
{
    try
    {
        while (!cancellationToken.IsCancellationRequested)
        {
            // Simulate time consuming Taks
            Console.WriteLine("Working...");
            await Task.Delay(1000); // Simulate Time consuming Task
            cancellationToken.ThrowIfCancellationRequested();
        }
    }
    catch (OperationCanceledException)
    {
        // Handle cancellation if needed
        Console.WriteLine("Operation cancelled.");
        // throw;               // we could throw the exception as well. More on this later in "ExceptionHandling" notebook.
    }
}

In [10]:
using System;
using System.Threading;
using System.Threading.Tasks;
using (var cancellationTokenSource = new CancellationTokenSource()){
    CancellationToken cancellationToken = cancellationTokenSource.Token;

    Task task = Task.Run(()=> PerformAsyncOperation(cancellationToken));

    Console.WriteLine("Press any key to cancel the task");
    Console.ReadLine();

    cancellationTokenSource.Cancel();

    await task; // awaiting to observe the exception it will raise.
}

Press any key to cancel the task
Working...
Working...
Operation cancelled.
